In [2]:
%matplotlib inline 
#有了matplotlib inline 就可以省掉plt.show()了
%load_ext autoreload
%autoreload 2

from __future__ import division
from imports import *
#import pylab #使用pylab模式，可以直接使用numpy、pandas和绘图，而不需要再次引入,会配置为使用你所指定的matplotlib gui后端，还会提供一种更接近于matlab的界面
#%pylab
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle code</button>""", raw=True)
# 允许多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toggle code

## 读写文本格式的数据

#### pandas中的将表格型数据读取为DataFrame对象的函数
* read_csv 从文件、url、文件型对象中加载带有分隔符的数据，默认为逗号
* read_table 从文件、url、文件型对象中加载带有分隔符的数据，默认为制表符（‘\t’）
* read_fwf 读取定宽列格式数据（也就是说，没有分隔符）
* read_clipboard 读取剪贴板中的数据（直接使用即可，亲测可用）
#### 这些函数完成了以下工作：
- 索引
- 类型推断和数据转换：你不需要指定列的类型
* 日期解析
* 迭代：支持对大文件进行逐块迭代
* 不规整数据问题：跳过一些行、页脚、注释

In [21]:
# 当文件没有标题行的时候，可以指定header=None，让pandas为其分配默认的列名
pd.read_csv('./data/ch06/ex2.csv')
pd.read_csv('./data/ch06/ex2.csv',header=None)
# 也可以自己定义列名
pd.read_csv('./data/ch06/ex2.csv',names=['a','b','c','d','message'])
# 如果希望将一列做成索引列，那么可以指定index_col='列名'
pd.read_csv('./data/ch06/ex2.csv',names=['a','b','c','d','message'],index_col='message')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [31]:
pd.read_table('./data/ch06/csv_mindex.csv')
# 将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可
pd.read_csv('./data/ch06/csv_mindex.csv',index_col=[0,1])

,"key1,key2,value1,value2"
0,"one,a,1,2"
1,"one,b,3,4"
2,"one,c,5,6"
3,"one,d,7,8"
4,"two,a,9,10"
5,"two,b,11,12"
6,"two,c,13,14"
7,"two,d,15,16"


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [34]:
list(open('./data/ch06/ex3.txt'))
# 对于一些表格不是由固定的分隔符去分隔字段的，可以编写一个正则表达式来作为read_table的分隔符
result = pd.read_table('./data/ch06/ex3.txt',sep='\s+') # \s匹配任意的空白符
result

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [37]:
# 还可以用skipprows跳过文件的第一行、第三行等
list(open('./data/ch06/ex4.csv'))
pd.read_csv('./data/ch06/ex4.csv',skiprows=[0,2,3])

['# hey!\n',
 'a,b,c,d,message\n',
 '# just wanted to make things more difficult for you\n',
 '# who reads CSV files with computers, anyway?\n',
 '1,2,3,4,hello\n',
 '5,6,7,8,world\n',
 '9,10,11,12,foo']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [50]:
# 对于缺失值，pandas会用一组经常出现的标记值进行识别，如NA、-1、NULL等
list(open('./data/ch06/ex5.csv'))
result = pd.read_csv('./data/ch06/ex5.csv')
pd.isnull(result)
# na_values 可以接受一组用于表示缺失值的字符串
#result = pd.read_csv('./data/ch06/ex5.csv',na_values=['lalallal'])
result
# 用字典指定哪些值变成NA
sentinels = {'message':['foo','world'],'something':['two']}
pd.read_csv('./data/ch06/ex5.csv',na_values=sentinels)

['something,a,b,c,d,message\n',
 'one,1,2,3,4,NA\n',
 'two,5,6,,8,world\n',
 'three,9,10,11,12,foo']

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,NaN
2,three,9,10,11.0,12,NaN


#### read_csv/read_table 函数的参数
* path 表示文件系统的位置、url、文件型对象的字符串
* sep/delimiter 用于对行中各字段进行拆分的字符序列或正则表达式
* header 用作列名的行号。默认为0，如果没有就应该设置为header=None
* index_col 用作行索引的列编号或列名。可以是列表
* names 用于结果的列名列表
* skiprows 需要忽略的行数（从文件开始处算起），需要跳过的行号列表
* na_values 一组用于替换成NA的值
* comment 用于将注释信息从行尾拆分出去的字符
* parse_dates 尝试将数据解析为日期，默认为False。如果为True，则尝试解析所有列
* keep_date_col 如果连接多列解析日期，则保持参与连接的列。默认为False
* converters 由列号/列名跟函数之间的映射关系组成的字典，例如{'foo':f}会对foo列的所有值应用函数f
* dayfirst 当解析有歧义的日期时，将其看作国际格式
* date_parser 用于解析日期的函数
* nrows 需要读取的行数（从文件开始处算起）
* iterator 返回一个TextParser以便逐块读取文件
* chunksize 文件块的大小
* skip_footer 需要忽略的行数（从文件末尾处算起）
* verbose 打印各种解析器输出信息
* encoding 用于unicode的文本编码格式，例如utf-8
* squeeze 如果数据经解析后仅含一列，则返回Series
* thousands 千分位分隔符

### 逐块读取文本文件

In [75]:
# 如果只想读取几行，通过设置nrows即可
pd.read_csv('./data/ch06/ex6.csv',nrows=5)
# 要逐块读取文件，需要设置chunksize（行数）
chunker = pd.read_csv('./data/ch06/ex6.csv',chunksize=1000)
# 返回的这个TextParser对象使你可以根据chunksize对文件进行逐块迭代
# 我们迭代处理ex6.csv,将值计数聚合到“key”列中
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 将数据写出到文本格式

In [92]:
# 利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中
data = pd.read_csv('./data/ch06/ex5.csv')
data
data.to_csv('./out.csv')
list(open('./out.csv'))
# 还可以使用其他分隔符，用sep=‘’
import sys
data.to_csv(sys.stdout,sep='|')
# 缺失值可以用na_rep=‘’表示为别的字符
data.to_csv(sys.stdout,na_rep='lalal')
# 默认会写出行和列的标签，也可以设置index=False,header=False来禁用
data.to_csv(sys.stdout,index=False,header=False)
# 还可以只写出一部分的列，并以你指定的顺序排列
data.to_csv(sys.stdout,columns=['a','b','c'])
# Series也有一个to_csv方法
dates = pd.date_range('1/1/2000',periods=7)
ts = Series(np.arange(7),index=dates)
ts.to_csv(sys.stdout)
# 用from_csv 可以将csv文件读取为Series
Series.from_csv('./data/ch06/tseries.csv')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


[',something,a,b,c,d,message\n',
 '0,one,1,2,3.0,4,\n',
 '1,two,5,6,,8,world\n',
 '2,three,9,10,11.0,12,foo\n']

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo
,something,a,b,c,d,message
0,one,1,2,3.0,4,lalal
1,two,5,6,lalal,8,world
2,three,9,10,11.0,12,foo
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo
,a,b,c
0,1,2,3.0
1,5,6,
2,9,10,11.0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

#### 手工处理分隔符格式

In [114]:
#pd.read_csv('./data/ch06/ex7.csv')
#list(open('./data/ch06/ex7.csv'))
# 对于单字符分隔符文件，可以使用python内置的csv模块，将文件传给csv.reader
import csv
f = open('./data/ch06/ex7.csv')
reader = csv.reader(f)

for line in reader:
    print line
    
lines = list(csv.reader(open('./data/ch06/ex7.csv')))
header,values = lines[0],lines[1:]
data_dict = {h:v for h,v in zip(header,zip(*values))}
data_dict
lines

# 通过定义csv.Dialect 的一个子类既可定义出新格式
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f,dialect=my_dialect)
reader = csv.reader(f,delimiter='|')
reader

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]

#### csv语支选项
* delimiter 用于分隔字段的单字符字符串，默认为‘，’
* lineterminator 用于写操作的行结束符，默认为‘\r\n’
* quotechar 用于带有特殊字符的字段的引用符号，默认为‘“’
* quoting 引用约定。可选值包括csv.QUOTE_ALL（引用所有字符），csv.QUOTE_MINIMAL（只引用带有诸如分隔符之类特殊字符的字段），csv.QUOTE_NON（不引用）
* skipinitialspace 忽略分隔符后面的空白符。默认为False
* doublequote 如何处理字段内的引用符号，如果为True，则双写
* escapechar 用于对分隔符进行转义的字符串。默认禁用

### JSON

In [136]:
obj = """
{"name":"Wes",
"place_lived":["united","spain","germany"],
"pet":null,
"siblings":[{"name":"scott","age":25,"pet":"zubo"},
            {"name":"Katie","age":33,"pet":"Cisco"}]
}
"""
# json.loads可以将JSON字符串转成Python形式
import json
result = json.loads(obj)
result
# json.dumps则将Python对象转换成Json格式
asjson = json.dumps(result)
asjson
# 转换为DataFrame的最简单的方式就是：向DataFrame构造器中传入一组JSON对象，并选取数据字段的子集

DataFrame(result['siblings'],columns=['name','age','pet'])

# 也可以用json_
from pandas.io.json import json_normalize 
json_normalize(result,'siblings')

{u'name': u'Wes',
 u'pet': None,
 u'place_lived': [u'united', u'spain', u'germany'],
 u'siblings': [{u'age': 25, u'name': u'scott', u'pet': u'zubo'},
  {u'age': 33, u'name': u'Katie', u'pet': u'Cisco'}]}

'{"pet": null, "place_lived": ["united", "spain", "germany"], "name": "Wes", "siblings": [{"pet": "zubo", "age": 25, "name": "scott"}, {"pet": "Cisco", "age": 33, "name": "Katie"}]}'

,name,age,pet
0,scott,25,zubo
1,Katie,33,Cisco


,age,name,pet
0,25,scott,zubo
1,33,Katie,Cisco


### XML和HTML：web信息收集

In [145]:
from lxml.html import parse
from urllib2 import urlopen
parsed = parse(urlopen('https://stackoverflow.com/questions'))
doc = parsed.getroot()
links = doc.findall('.//a')
links[0:10]
lnk = links[10]
# get适用于URL
lnk.get('href')
lnk.text_content()

urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

[<Element a at 0xb8470c0>,
 <Element a at 0xb81cea0>,
 <Element a at 0xb81cb40>,
 <Element a at 0xb84ac00>,
 <Element a at 0xb84ac60>,
 <Element a at 0xb84acc0>,
 <Element a at 0xb84ac90>,
 <Element a at 0xb84acf0>,
 <Element a at 0xb84ad20>,
 <Element a at 0xb84ad50>]

'https://stackexchange.com/users/?tab=reputation'

'\r\n            \r\n            \r\n        '

['https://stackapps.com',
 'https://api.stackexchange.com',
 'https://data.stackexchange.com',
 'https://area51.stackexchange.com',
 'https://stackoverflow.blog?blb=1',
 'https://www.facebook.com/officialstackoverflow/',
 'https://twitter.com/stackoverflow',
 'https://linkedin.com/company/stack-overflow',
 'https://creativecommons.org/licenses/by-sa/3.0/',
 'https://stackoverflow.blog/2009/06/25/attribution-required/']

In [171]:
# 获取到数据的url，利用urllib2的urlopen将连接打开
parsed = parse(urlopen('http://www.boyaa.com.hk/s/ir_highlight.php'))
# 利用lxml解析得到所有节点
doc = parsed.getroot()
# 通过这个对象，你可以获取特定类型的html标签
tables = doc.findall('.//table')
tables
calls = tables[0]
second_table = tables[1]
calls
second_table
rows = calls.findall('.//tr')

def _unpack(row):
    elts = row.findall('.//td')
    # text_content方法适用于文本
    return [val.text_content() for val in elts]

#_unpack(rows[2])

# TextParser类可用于自动类型转换
from pandas.io.parsers import TextParser

def parser_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[1])
    data = [_unpack(r) for r in rows[2:]]
    return TextParser(data,names=header).get_chunk()

call_data = parser_options_data(calls)
call_data
second_data = parser_options_data(second_table)
second_data

[<Element table at 0xba9c7b0>, <Element table at 0xbab5bd0>]

<Element table at 0xba9c7b0>

<Element table at 0xbab5bd0>

,(人民币千元),2017,2016,2015,2014,2013
0,收益,"735,602","745,194","813,480","945,319","681,262"
1,毛利,"457,267","471,708","427,944","567,852","416,209"
2,经营利润,"230,475","211,023","377,736","326,649","176,191"
3,本公司拥有人应占利润,"243,245","211,271","357,799","280,065","135,507"
4,非国际财务报告准则计量：经调整纯利（未经审计）,"253,110","232,817","379,726","306,177","218,676"
5,毛利率,62.2%,63.3%,52.6%,60.1%,61.1%
6,经营利润率,31.3%,28.3%,46.4%,34.6%,25.9%
7,纯利率,33.1%,28.4%,44.0%,29.6%,19.9%
8,非国际财务报告准则计量：经调整纯利率（未经审计）,34.4%,31.2%,46.7%,32.4%,32.1%


,(人民币千元),2017,2016,2015,2014,2013
0,总资产,"2,506,073","2,284,954","2,079,800","1,713,681","1,408,113"
1,总负债,"281,912","263,389","273,496","232,055","193,494"
2,总权益,"2,224,161","2,021,565","1,806,304","1,481,626","1,214,619"


## 二进制数据格式

In [178]:
# 实现数据的二进制格式存储最简单的办法之一是适用Python内置的pickle序列化
frame = pd.read_csv('./data/ch06/ex1.csv')
frame
# 用to_pickle方法保存
frame.to_pickle('./data/ch06/frame_pickle')
# read_pickle方法读取
pd.read_pickle('./data/ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


#### HDF5格式
HDF5是一个能实现高效读写磁盘上以二进制格式存储的科学数据，是一个C库，带有许多语言的接口
HDF指的是层次型数据格式（hierarchical data format）
每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据
HDF5zhichi多种压缩器的即时压缩，还能更高效地存储重复模式数据
对于那些非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写

#### python中的HDF5库有两个接口
#### h5py提供了一种直接而高级的HDF5API访问接口
#### PyTables则抽象了HDF5的许多细节以提供多种灵活的数据容器、表索引、查询功能以及对核外计算技术的某些支持

### 读取excel文件

In [182]:
# 先安装xlrd包
xls_file = pd.ExcelFile('./data/ch06/ex1.xlsx')
xls_file
# 工作表中的数据可以通过parse读取到DataFrame中
table = xls_file.parse('Sheet1')
table

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 使用HTML和Web API

In [201]:
import requests
url = 'http://bud_server.oa.com/new/?charttype=table&dates=%7B%22sdate%22%3A%222018-05-27%22%2C%22edate%22%3A%222018-06-10%22%7D&dims=%5B%22dsu%22%2C%22pa_system_dsu%22%5D'
resp = requests.get(url)
resp
# Response对象的text属性含有get请求的内容
data = json.loads(resp.text)
keys = data.keys()
keys
dict_you_want={'my_items':data['data']for key in keys}
dict_you_want
# 用一个列表定义出感兴趣的字段，然后将列表传给DataFrame
list1 = ['displayName','attachmentData','data','sqls']
bud = DataFrame(dict_you_want,columns=list1)
bud

<Response [200]>

[u'data', u'error_message', u'error_code']

{'my_items': {}}

,displayName,attachmentData,data,sqls


## 使用数据库

In [215]:
import sqlite3
query = """
CREATE TABLE test
( a VARCHAR(20),
b VARCHAR(20),
c REAL,
d INTEGER);
"""
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

data = [('Atlandta','Georgia',1.23,6),
       ('Tallahass','Floradia',2.6,3)]
stmt = "INSERT INTO test VALUES(?,?,?,?)"
con.executemany(stmt,data)
con.commit

cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows
cursor.description
DataFrame(rows,columns=zip(*cursor.description)[0])

# pandas 有一个read_frame函数
import pandas.io.sql as sql
sql.frame_query('select * from test',con)

<function commit>

[(u'Atlandta', u'Georgia', 1.23, 6), (u'Tallahass', u'Floradia', 2.6, 3)]

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

,a,b,c,d
0,Atlandta,Georgia,1.23,6
1,Tallahass,Floradia,2.60,3


AttributeError: 'module' object has no attribute 'frame_query'

#### 存取MongoDB中的数据